In [1]:
import sys
sys.path.append('../harmonizome')

import pandas as pd
import numpy as np
import utility_functions as uf
import timeit
import itertools
import scipy.spatial.distance as dist

In [2]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

### removeAndInpute

In [3]:
df = pd.DataFrame(data=np.random.randint(10,size=(5, 10)))
df = df.mask(np.random.random(df.shape) < .1)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,2,1,7.0,3,6,NaN,4.0,2,8.0,3.0
1,6,8,5.0,0,4,3.0,NaN,5,8.0,NaN
2,5,8,2.0,7,2,7.0,1.0,0,6.0,6.0
3,3,0,NaN,2,2,2.0,5.0,1,2.0,7.0
4,9,8,5.0,5,3,NaN,4.0,3,NaN,8.0


In [4]:
df.shape

(5, 10)

## makeBinaryMatrix

In [31]:
df = pd.DataFrame(data=np.random.randint(3000,size=(7500, 2)), columns=['Gene Symbol', 'Attributes'])
df = df.set_index('Gene Symbol')
df.head()

,Attributes
Gene Symbol,
1460,2085
2012,395
958,2260
421,2326
1230,2555


In [32]:
%mprun -f uf.createBinaryMatrix binary_matrix = uf.createBinaryMatrix(df)

Filename: ../harmonizome/utility_functions.py

Line #    Mem usage    Increment   Line Contents
   404    883.2 MiB    883.2 MiB   def createBinaryMatrix(df):
   405                                 '''
   406                                 Creates an adjacency matrix from df, which is a gene-attribute edge
   407                                 list.
   408                                 '''
   409    883.2 MiB      0.0 MiB       matrix = pd.crosstab(df.index, df.iloc[:, 0]) > 0
   410    883.2 MiB      0.0 MiB       matrix.index.name = df.index.name
   411    883.2 MiB      0.0 MiB       matrix.columns.name = df.columns[0]
   412    883.2 MiB      0.0 MiB       return matrix

In [33]:
%%memit
binary_matrix.shape

peak memory: 883.25 MiB, increment: 0.00 MiB


In [41]:
%mprun -f uf.createSimilarityMatrix similarityuf = uf.createSimilarityMatrix(binary_matrix, 'jaccard', sparse=True)

Filename: ../harmonizome/utility_functions.py

Line #    Mem usage    Increment   Line Contents
   202    883.4 MiB    883.4 MiB   def createSimilarityMatrix(inputDF, metric, dtype=None, sparse=False):
   203                                 '''
   204                                 Creates a similarity matrix between the rows of the inputDF based on
   205                                 the metric specified. The resulting matrix has both rows and columns labeled
   206                                 by the index of inputDF.
   207                                 '''
   208    883.4 MiB      0.0 MiB       if sparse and metric == 'jaccard':
   209                                     # from na-o-ys on Github
   210    883.4 MiB      0.0 MiB           sparse = sp.csr_matrix(inputDF.to_numpy(dtype=np.bool).astype(np.int))
   211    883.4 MiB      0.0 MiB           cols_sum = sparse.getnnz(axis=1)
   212    883.4 MiB      0.0 MiB           ab = sparse * sparse.T
   213    883.4 MiB      0

In [35]:
%mprun -f uf.createSimilarityMatrix similarity_matrix = uf.createSimilarityMatrix(binary_matrix, 'jaccard')

Filename: ../harmonizome/utility_functions.py

Line #    Mem usage    Increment   Line Contents
   202    883.2 MiB    883.2 MiB   def createSimilarityMatrix(inputDF, metric, dtype=None, sparse=False):
   203                                 '''
   204                                 Creates a similarity matrix between the rows of the inputDF based on
   205                                 the metric specified. The resulting matrix has both rows and columns labeled
   206                                 by the index of inputDF.
   207                                 '''
   208    883.2 MiB      0.0 MiB       if sparse and metric == 'jaccard':
   209                                     # from na-o-ys on Github
   210                                     sparse = sp.csr_matrix(inputDF.to_numpy(dtype=np.bool).astype(np.int))
   211                                     cols_sum = sparse.getnnz(axis=1)
   212                                     ab = sparse * sparse.T
   213                    

In [42]:
np.allclose(similarityuf, similarity_matrix)

True

In [52]:
df = pd.DataFrame(data=np.random.randint(3000,size=(10000, 10000)))
np.log2(df + 1)

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,8.124121,11.361396,8.257388,11.506308,11.460456,8.189825,9.938109,11.401946,9.533330,11.494856,...,10.665336,8.442943,10.202124,11.255619,10.195987,10.729621,10.343186,9.523562,10.253847,6.285402
1,10.446049,11.027215,3.459432,10.854868,11.445015,4.584963,9.573647,9.789534,11.201511,10.659996,...,11.510270,5.643856,6.266787,11.126059,7.734710,9.321928,11.415742,8.262095,11.418380,10.281930
2,11.092096,10.373953,9.800900,7.577429,7.700440,11.329796,11.240195,9.319672,9.982994,8.599913,...,10.763212,8.658211,10.162391,10.183635,10.045760,10.126704,10.821774,10.711667,9.800900,11.429930
3,11.086800,11.179909,11.419960,9.741467,10.378295,8.348728,7.523562,11.243174,11.210671,8.797662,...,10.689124,9.942515,10.871905,10.818582,9.994353,10.702173,11.200899,11.062721,10.675957,11.307770
4,5.930737,11.354800,10.842350,9.995767,10.043027,11.455327,9.744834,10.473706,7.948367,8.679480,...,11.335390,11.348728,11.220378,10.926296,11.518653,7.584963,8.592457,8.861087,7.199672,11.379920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,11.164907,10.310613,11.499348,10.970825,11.438272,5.727920,10.725366,10.620220,10.449149,8.228819,...,10.609179,10.109831,9.511753,8.758223,11.084144,10.958553,5.727920,5.672425,11.474720,8.581201
9996,9.129283,10.755722,10.617467,9.383704,11.401413,10.356452,6.714246,10.556506,10.473706,11.474213,...,10.980140,9.859535,10.435670,11.342630,10.767357,9.481799,10.995767,11.404609,7.417853,9.867279
9997,8.794416,8.108524,10.511753,10.829723,7.977280,11.401946,10.741467,10.001408,8.447083,11.209453,...,8.754888,10.100662,11.423641,11.284824,10.428360,11.144658,8.303781,9.722808,10.694358,9.113742
9998,11.501340,11.492354,10.704768,11.429930,8.861087,10.986553,10.825754,11.194141,10.908393,11.280191,...,9.776433,11.443980,10.084808,11.309476,10.990104,11.457381,8.562242,10.831307,11.481295,10.911392


In [53]:
uf.zscore(df)

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,-1.082608,1.026998,-1.058391,1.276348,1.194726,-1.070948,-0.452955,1.094269,-0.668221,1.255718,...,0.123778,-1.020719,-0.276258,0.860167,-0.280743,0.190152,-0.167728,-0.672706,-0.237689,-1.262894
1,-0.071524,0.554991,-1.324555,0.342230,1.189655,-1.312785,-0.644623,-0.533262,0.797630,0.130373,...,1.306448,-1.289246,-1.264801,0.688986,-1.141670,-0.755078,1.138954,-1.056566,1.143481,-0.207330
2,0.632372,-0.134900,-0.524810,-1.153149,-1.137911,0.982843,0.843909,-0.751586,-0.417249,-0.976568,...,0.233498,-0.962227,-0.297139,-0.281901,-0.376913,-0.322236,0.298035,0.178821,-0.524810,1.148667
3,0.633299,0.767550,1.156418,-0.587931,-0.148140,-1.078644,-1.210119,0.863841,0.813844,-0.968465,...,0.148140,-0.469419,0.354610,0.291651,-0.436087,0.162028,0.799030,0.599967,0.134252,0.966614
4,-1.298163,1.024005,0.313193,-0.426669,-0.395803,1.195581,-0.574641,-0.062639,-1.129311,-0.981338,...,0.992232,1.014020,0.812487,0.413052,1.309964,-1.179240,-1.003126,-0.931409,-1.220092,1.065764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.717496,-0.201544,1.254050,0.458624,1.146560,-1.291672,0.177359,0.070764,-0.086888,-1.070422,...,0.060015,-0.349343,-0.685249,-0.951287,0.605527,0.443396,-1.291672,-1.293463,1.210159,-0.996074
9996,-0.841225,0.203825,0.062578,-0.744677,1.076339,-0.169854,-1.247981,0.004470,-0.070624,1.201495,...,0.463970,-0.511351,-0.103700,0.979791,0.216341,-0.702660,0.483638,1.081703,-1.188979,-0.506881
9997,-0.964353,-1.117482,-0.038282,0.289853,-1.139358,1.097430,0.191412,-0.434779,-1.050944,0.789347,...,-0.975291,-0.368241,1.134801,0.905106,-0.113024,0.694553,-1.081023,-0.598847,0.141280,-0.864089
9998,1.259214,1.243291,0.171168,1.135371,-0.893878,0.489621,0.300318,0.767382,0.394970,0.894763,...,-0.529427,1.159255,-0.344548,0.939877,0.494044,1.182255,-0.970838,0.306510,1.223830,0.398508
